In [14]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import mean_squared_error as mse


In [15]:
train = pd.read_csv('../../data/engineered/training.csv', index_col='id')
submission_input = pd.read_csv('../../data/engineered/test.csv', index_col='id')

target = 'SalePrice'
y = train[target]
X = train[[col for col in train.columns if col != target]]
y_log = np.log1p(y)

In [16]:
# with open('lasso.pickle', "wb") as pickle_model:
#     pickle.dump(gs_las_rg, pickle_model)


with open('lasso.pickle', "rb") as pickle_model:
    lasso_model = pickle.load(pickle_model)

In [17]:
# with open('ridge.pickle', "wb") as pickle_model:
#     pickle.dump(gs_rid_rg, pickle_model)


with open('ridge.pickle', "rb") as pickle_model:
    ridge_model = pickle.load(pickle_model)

In [18]:
# with open('elastic_net.pickle', "wb") as pickle_model:
#     pickle.dump(gs_ela_rg, pickle_model)


with open('elastic_net.pickle', "rb") as pickle_model:
    elastic_model = pickle.load(pickle_model)

In [19]:
mse(np.exp(elastic_model.predict(X))-1,y)**.5

29589.826665435201

In [20]:
mse(np.exp(ridge_model.predict(X))-1,y)**.5

29267.654480737267

In [21]:
mse(np.exp(lasso_model.predict(X))-1,y)**.5

29125.707090074819

In [22]:
ensemble = (np.exp(ridge_model.predict(X))-1 + np.exp(elastic_model.predict(X))-1 + np.exp(lasso_model.predict(X))-1)/3

In [23]:
mse(ensemble, y) **.5

29117.016048238711

In [24]:
submit = (np.exp(ridge_model.predict(submission_input))-1 + np.exp(elastic_model.predict(submission_input))-1 \
          + np.exp(lasso_model.predict(submission_input))-1)/3
submit = pd.DataFrame(submit, index=submission_input.index, columns=['SalePrice'])

In [25]:
submit.sort_index().to_csv('ensemble.csv')

In [41]:
ensemble_prod = (ridge_model.predict(X) + lasso_model.predict(X) + elastic_model.predict(X))/3


In [42]:
ensemble_prod = np.exp(ensemble_prod) - 1

In [43]:
mse(ensemble_prod, y)**.5

29100.480228216988

In [44]:
ensemble_prod_submit = (ridge_model.predict(submission_input)\
                        + lasso_model.predict(submission_input) \
                        + elastic_model.predict(submission_input))/3


In [45]:
ensemble_prod_submit = np.exp(ensemble_prod_submit) - 1

In [47]:
submit_prod = pd.DataFrame(ensemble_prod_submit, index=submission_input.index, columns=['SalePrice'])

In [48]:
submit_prod.to_csv('ensemble_product.csv')